In [264]:
import pandas as pd
import glob
import random
import polars as pl

import hashlib
import os

from datetime import datetime, timedelta

In [324]:
energy_type_dict = {
    'Electric': 'E',
    'Gas': 'G',
    'Water': 'W'
}

energy_units_dict = {
    '1' : 'kWh',
    '101' : 'kWh',
    '2' : 'kWh',
    '102' : 'kWh',
    '21': 'kVARh',
    '121': 'kVARh',
    '31': 'kCCF', # Validate this
    '131': 'kCCF' # Validate this
}


energy_directions_dict = {
    '1' : 'delivered',
    '101' : 'delivered',
    '2' : 'received',
    '102' : 'received',
    '21': 'delivered',
    '121': 'delivered',
    '31': 'N/A', # Validate this
    '131': 'N/A' # Validate this
    
}

channel_id_dict = {
    '1' : 'usage_delivered',
    '101' : 'usage_delivered',
    '2' : 'usage_received',
    '102' : 'usage_received',
    '21': 'reactive_delivered',
    '121': 'reactive_delivered',
    '31': 'N/A', # Validate this
    '131': 'N/A' # Validate this
}
 
unit_to_kilounit = lambda x: x / 1_000
value_tranformation_dict = {
    '1' : unit_to_kilounit,
    '101' : unit_to_kilounit,
    '2' : unit_to_kilounit,
    '102' : unit_to_kilounit,
    '21': unit_to_kilounit,
    '121': unit_to_kilounit,
    '31': lambda x: x , # By the moment Identity
    '131':lambda x: x , # By the moment Identity
}

opco_dict ={
    '1160': 'NYSEG', 
    '1000': 'RGE',
    'MISSING': 'CMP'
}

df_approved_channels = pl.DataFrame({'channel_id':list(energy_units_dict.keys())})

list_final_order_file = ["service_point_id","meter_id","timestamp","timezone","interval_value","interval_units","usage","energy_type","energy_units","energy_direction","is_estimate","is_outage","channel_id","is_deleted","update_datetime"]

df_readings_code = pl.read_parquet('C:\\Users\sqyan\\NTT DATA EMEAL\IEDR - Documents\IEDR - IT\\000 - Data Specifications (NYSERDA)\Customer Data\Sample Data Delivery August 1st 2023\MeterReadingSync_parquet\STATUSCODES-AMI.parquet').select('ReadingStatusRef','SourceValidation','StatusCodes').unique()

In [327]:
df_readings_code = df_readings_code.with_columns(
    pl.col('StatusCodes').apply(lambda x: any([s.startswith('EST') for s in x.split(',')])).alias('is_estimate'), # if the code starts with EST, then is_estimated = True else is_estimated = False
    pl.col('StatusCodes').apply(lambda x: any([s.startswith('POWER') for s in x.split(',')])).alias('is_outage'), # if the code starts with POWER, then is_outage = True else is_outage = False
)
df_readings_code.head()

ReadingStatusRef,SourceValidation,StatusCodes,is_estimate,is_outage
str,str,str,bool,bool
"""84""","""NV""","""IRREGULAR,NV,V…",false,false
"""108""","""NV""","""NV,TIMERESET,V…",false,false
"""146""","""NV""","""IRREGULAR,NV,O…",false,false
"""71""","""NV""","""ESTIMATED,ESTU…",true,false
"""82""","""NV""","""NV,POWER,VEESE…",false,true


In [5]:
input_parquet = 'C:\\Users\sqyan\\NTT DATA EMEAL\IEDR - Documents\IEDR - IT\\000 - Data Specifications (NYSERDA)\Customer Data\Sample Data Delivery August 1st 2023\MeterReadingSyncSave_manual_output_new_\*.parquet'
input_parquet = 'C:\\Users\sqyan\Documents\Customer Data - AMI\MeterReadingSyncSave_parquet\*.parquet'
input_parquet = 'C:\\Users\sqyan\\NTT DATA EMEAL\IEDR - Documents\IEDR - IT\\000 - Data Specifications (NYSERDA)\Customer Data\Sample Data Delivery August 1st 2023\MeterReadingSync_parquet\AMI-*.parquet'
files_path = glob.glob(input_parquet)
len(files_path)

197

In [106]:
df_5_min_nyseg = pl.read_csv('approved_meters/new_5min_approved_NYSEG.csv')
df_5_min_rge = pl.read_csv('approved_meters/new_5min_approved_RGE.csv')

In [107]:
df_5_min_nyseg = df_5_min_nyseg.with_columns(pl.all().cast(pl.Utf8, strict=False))
df_5_min_nyseg = df_5_min_nyseg.with_columns(
    pl.col('ApprovedDate').str.to_datetime('%m/%d/%Y')
)

df_5_min_rge = df_5_min_rge.with_columns(pl.all().cast(pl.Utf8, strict=False))
df_5_min_rge = df_5_min_rge.with_columns(
    pl.col('ApprovedDate').str.to_datetime('%m/%d/%Y')
)

In [108]:
df_gas_nyseg = pl.read_csv('approved_meters/new_gas_approved_NYSEG.csv')
df_gas_rge = pl.read_csv('approved_meters/new_gas_approved_RGE.csv')

In [109]:
df_gas_nyseg = df_gas_nyseg.with_columns(pl.all().cast(pl.Utf8, strict=False))
df_gas_nyseg = df_gas_nyseg.with_columns(
    pl.col('ApprovedDate').str.to_datetime('%m/%d/%Y')
)

df_gas_rge = df_gas_rge.with_columns(pl.all().cast(pl.Utf8, strict=False))
df_gas_rge = df_gas_rge.with_columns(
    pl.col('ApprovedDate').str.to_datetime('%m/%d/%Y')
)

In [313]:
df_approved_meters = pl.concat([df_gas_rge, df_gas_nyseg])
df_approved_meters = df_approved_meters.select(['Meterid', 'ApprovedDate','CoCd'])
df_approved_meters.head()

Meterid,ApprovedDate,CoCd
str,datetime[μs],str
"""103253337""",2023-07-11 00:00:00,"""1000"""
"""101580563""",2023-06-23 00:00:00,"""1000"""
"""101680057""",2023-06-23 00:00:00,"""1000"""
"""102588940""",2023-06-23 00:00:00,"""1000"""
"""101465034""",2023-06-23 00:00:00,"""1000"""


In [274]:
def filter_dataframe(df_exp):
    df_final = df_exp.with_columns(
        pl.col('Value').cast(pl.Float64)
        # Append the StatusCodes to calculate if is estimate or is outage
    ).join(df_readings_code.select('ReadingStatusRef','is_estimate', 'is_outage'), left_on='StatusRef', right_on='ReadingStatusRef', how='left'
          ).with_columns(
        pl.col('Meterid').alias('service_point_id'),
        pl.col('Meterid').alias('meter_id'),
        pl.col('ReadingTime').dt.replace_time_zone("UTC").dt.convert_time_zone(time_zone='US/Eastern').dt.strftime('%Y-%m-%dT%H:%M:%S.000%z').alias('timestamp'),
        pl.col('ReadingTime').dt.replace_time_zone("UTC").dt.convert_time_zone(time_zone='US/Eastern').dt.strftime('%Z').alias('timezone'),
        pl.col('IntervalLength').alias('interval_value'),
        pl.lit('minute').alias("interval_units"),
        pl.col('Value').mul(0.001).alias('usage'), # Transform to Kilo
        pl.col('MarketType').apply(lambda x: energy_type_dict.get(x, None)).alias('energy_type'),
        pl.col('channel_id').apply(lambda x: energy_units_dict.get(x, None)).alias('energy_units'),
        pl.col('channel_id').apply(lambda x: energy_directions_dict.get(x, None)).alias('energy_direction'),
        pl.col('is_estimate'), 
        pl.col('is_outage'),
        pl.col('channel_id').apply(lambda x: channel_id_dict.get(x, None)).alias('channel_id'),
        pl.lit('false').alias('is_deleted'),
        pl.col('ReadingTime').dt.replace_time_zone("UTC").dt.convert_time_zone(time_zone='US/Eastern').dt.strftime('%Y-%m-%dT%H:%M:%S.000%z').alias('update_datetime'), ## TODO: Missing from the xmls /Header/Creation_Datetime =>
        pl.col('CoCd') 
    )
    return df_final.with_columns(
        pl.col('timestamp').apply(lambda x: x[:-2] + ':' + x[-2:]), # Change from 0400 to 04:00
        pl.col('update_datetime').apply(lambda x: x[:-2] + ':' + x[-2:]) # Change from 0400 to 04:00
    ).select([
        "service_point_id",
        "meter_id",
        "timestamp",
        "timezone",
        "interval_value",
        "interval_units",
        "usage",
        "energy_type",
        "energy_units",
        "energy_direction",
        "is_estimate",
        "is_outage",
        "channel_id",
        "is_deleted",
        "update_datetime",
        "CoCd" # To split by opco during the generation of the TSV
    ])

In [226]:
dfs_back = dfs.copy()

In [316]:
dfs = []
for fn in files_path:
    print(fn)
    df = pl.read_parquet(fn)
    df = df.with_columns(
            pl.col('ReadingTime').str.to_datetime('%Y-%m-%dT%H:%M:%SZ')
        )

#     df_ = df_approved_meters.join(
#             df, left_on='Meterid', right_on='meter_num', how="inner"
#         ).join(df_approved_channels, on='channel_id', how="inner").filter(
#             pl.col('ReadingTime') >= pl.col('ApprovedDate')
#         )
    df_ = df_approved_meters.join(
            df, left_on='Meterid', right_on='meter_num', how="inner"
        ).join(df_approved_channels, on='channel_id', how="inner").filter(
            pl.col('ReadingTime') >= pl.col('ApprovedDate')
        )
    
    if len(df_) > 0:
        dfs.append(df_)

C:\Users\sqyan\NTT DATA EMEAL\IEDR - Documents\IEDR - IT\000 - Data Specifications (NYSERDA)\Customer Data\Sample Data Delivery August 1st 2023\MeterReadingSync_parquet\AMI-0-0.parquet
C:\Users\sqyan\NTT DATA EMEAL\IEDR - Documents\IEDR - IT\000 - Data Specifications (NYSERDA)\Customer Data\Sample Data Delivery August 1st 2023\MeterReadingSync_parquet\AMI-0-1.parquet
C:\Users\sqyan\NTT DATA EMEAL\IEDR - Documents\IEDR - IT\000 - Data Specifications (NYSERDA)\Customer Data\Sample Data Delivery August 1st 2023\MeterReadingSync_parquet\AMI-10000-0.parquet
C:\Users\sqyan\NTT DATA EMEAL\IEDR - Documents\IEDR - IT\000 - Data Specifications (NYSERDA)\Customer Data\Sample Data Delivery August 1st 2023\MeterReadingSync_parquet\AMI-10000-1.parquet
C:\Users\sqyan\NTT DATA EMEAL\IEDR - Documents\IEDR - IT\000 - Data Specifications (NYSERDA)\Customer Data\Sample Data Delivery August 1st 2023\MeterReadingSync_parquet\AMI-10400-0.parquet
C:\Users\sqyan\NTT DATA EMEAL\IEDR - Documents\IEDR - IT\000 - 

C:\Users\sqyan\NTT DATA EMEAL\IEDR - Documents\IEDR - IT\000 - Data Specifications (NYSERDA)\Customer Data\Sample Data Delivery August 1st 2023\MeterReadingSync_parquet\AMI-17600-1.parquet
C:\Users\sqyan\NTT DATA EMEAL\IEDR - Documents\IEDR - IT\000 - Data Specifications (NYSERDA)\Customer Data\Sample Data Delivery August 1st 2023\MeterReadingSync_parquet\AMI-18000-0.parquet
C:\Users\sqyan\NTT DATA EMEAL\IEDR - Documents\IEDR - IT\000 - Data Specifications (NYSERDA)\Customer Data\Sample Data Delivery August 1st 2023\MeterReadingSync_parquet\AMI-18000-1.parquet
C:\Users\sqyan\NTT DATA EMEAL\IEDR - Documents\IEDR - IT\000 - Data Specifications (NYSERDA)\Customer Data\Sample Data Delivery August 1st 2023\MeterReadingSync_parquet\AMI-18400-0.parquet
C:\Users\sqyan\NTT DATA EMEAL\IEDR - Documents\IEDR - IT\000 - Data Specifications (NYSERDA)\Customer Data\Sample Data Delivery August 1st 2023\MeterReadingSync_parquet\AMI-18400-1.parquet
C:\Users\sqyan\NTT DATA EMEAL\IEDR - Documents\IEDR - I

C:\Users\sqyan\NTT DATA EMEAL\IEDR - Documents\IEDR - IT\000 - Data Specifications (NYSERDA)\Customer Data\Sample Data Delivery August 1st 2023\MeterReadingSync_parquet\AMI-26000-0.parquet
C:\Users\sqyan\NTT DATA EMEAL\IEDR - Documents\IEDR - IT\000 - Data Specifications (NYSERDA)\Customer Data\Sample Data Delivery August 1st 2023\MeterReadingSync_parquet\AMI-26400-0.parquet
C:\Users\sqyan\NTT DATA EMEAL\IEDR - Documents\IEDR - IT\000 - Data Specifications (NYSERDA)\Customer Data\Sample Data Delivery August 1st 2023\MeterReadingSync_parquet\AMI-26400-1.parquet
C:\Users\sqyan\NTT DATA EMEAL\IEDR - Documents\IEDR - IT\000 - Data Specifications (NYSERDA)\Customer Data\Sample Data Delivery August 1st 2023\MeterReadingSync_parquet\AMI-26800-0.parquet
C:\Users\sqyan\NTT DATA EMEAL\IEDR - Documents\IEDR - IT\000 - Data Specifications (NYSERDA)\Customer Data\Sample Data Delivery August 1st 2023\MeterReadingSync_parquet\AMI-26800-1.parquet
C:\Users\sqyan\NTT DATA EMEAL\IEDR - Documents\IEDR - I

C:\Users\sqyan\NTT DATA EMEAL\IEDR - Documents\IEDR - IT\000 - Data Specifications (NYSERDA)\Customer Data\Sample Data Delivery August 1st 2023\MeterReadingSync_parquet\AMI-34400-0.parquet
C:\Users\sqyan\NTT DATA EMEAL\IEDR - Documents\IEDR - IT\000 - Data Specifications (NYSERDA)\Customer Data\Sample Data Delivery August 1st 2023\MeterReadingSync_parquet\AMI-34400-1.parquet
C:\Users\sqyan\NTT DATA EMEAL\IEDR - Documents\IEDR - IT\000 - Data Specifications (NYSERDA)\Customer Data\Sample Data Delivery August 1st 2023\MeterReadingSync_parquet\AMI-34800-0.parquet
C:\Users\sqyan\NTT DATA EMEAL\IEDR - Documents\IEDR - IT\000 - Data Specifications (NYSERDA)\Customer Data\Sample Data Delivery August 1st 2023\MeterReadingSync_parquet\AMI-34800-1.parquet
C:\Users\sqyan\NTT DATA EMEAL\IEDR - Documents\IEDR - IT\000 - Data Specifications (NYSERDA)\Customer Data\Sample Data Delivery August 1st 2023\MeterReadingSync_parquet\AMI-35200-0.parquet
C:\Users\sqyan\NTT DATA EMEAL\IEDR - Documents\IEDR - I

C:\Users\sqyan\NTT DATA EMEAL\IEDR - Documents\IEDR - IT\000 - Data Specifications (NYSERDA)\Customer Data\Sample Data Delivery August 1st 2023\MeterReadingSync_parquet\AMI-6000-1.parquet
C:\Users\sqyan\NTT DATA EMEAL\IEDR - Documents\IEDR - IT\000 - Data Specifications (NYSERDA)\Customer Data\Sample Data Delivery August 1st 2023\MeterReadingSync_parquet\AMI-6400-0.parquet
C:\Users\sqyan\NTT DATA EMEAL\IEDR - Documents\IEDR - IT\000 - Data Specifications (NYSERDA)\Customer Data\Sample Data Delivery August 1st 2023\MeterReadingSync_parquet\AMI-6400-1.parquet
C:\Users\sqyan\NTT DATA EMEAL\IEDR - Documents\IEDR - IT\000 - Data Specifications (NYSERDA)\Customer Data\Sample Data Delivery August 1st 2023\MeterReadingSync_parquet\AMI-6800-0.parquet
C:\Users\sqyan\NTT DATA EMEAL\IEDR - Documents\IEDR - IT\000 - Data Specifications (NYSERDA)\Customer Data\Sample Data Delivery August 1st 2023\MeterReadingSync_parquet\AMI-6800-1.parquet
C:\Users\sqyan\NTT DATA EMEAL\IEDR - Documents\IEDR - IT\000

ArrowErrorException: ExternalFormat("File out of specification: The file must end with PAR1")

In [317]:
len(dfs)

147

In [328]:
df_finals = [filter_dataframe(df_) for df_ in dfs]
df_finals[0].head()

service_point_id,meter_id,timestamp,timezone,interval_value,interval_units,usage,energy_type,energy_units,energy_direction,is_estimate,is_outage,channel_id,is_deleted,update_datetime,CoCd
str,str,str,str,str,str,f64,str,str,str,bool,bool,str,str,str,str
"""101732788""","""101732788""","""2023-06-19T01:…","""EDT""","""60""","""minute""",0.0,"""G""","""kCCF""","""N/A""",false,false,"""N/A""","""false""","""2023-06-19T01:…","""1000"""
"""101732788""","""101732788""","""2023-06-19T02:…","""EDT""","""60""","""minute""",0.0,"""G""","""kCCF""","""N/A""",false,false,"""N/A""","""false""","""2023-06-19T02:…","""1000"""
"""101732788""","""101732788""","""2023-06-19T03:…","""EDT""","""60""","""minute""",0.0,"""G""","""kCCF""","""N/A""",false,false,"""N/A""","""false""","""2023-06-19T03:…","""1000"""
"""101732788""","""101732788""","""2023-06-19T04:…","""EDT""","""60""","""minute""",0.0,"""G""","""kCCF""","""N/A""",false,false,"""N/A""","""false""","""2023-06-19T04:…","""1000"""
"""101732788""","""101732788""","""2023-06-19T05:…","""EDT""","""60""","""minute""",0.0,"""G""","""kCCF""","""N/A""",false,false,"""N/A""","""false""","""2023-06-19T05:…","""1000"""


In [329]:
df_final = pl.concat(df_finals)
df_final.head()

service_point_id,meter_id,timestamp,timezone,interval_value,interval_units,usage,energy_type,energy_units,energy_direction,is_estimate,is_outage,channel_id,is_deleted,update_datetime,CoCd
str,str,str,str,str,str,f64,str,str,str,bool,bool,str,str,str,str
"""101732788""","""101732788""","""2023-06-19T01:…","""EDT""","""60""","""minute""",0.0,"""G""","""kCCF""","""N/A""",false,false,"""N/A""","""false""","""2023-06-19T01:…","""1000"""
"""101732788""","""101732788""","""2023-06-19T02:…","""EDT""","""60""","""minute""",0.0,"""G""","""kCCF""","""N/A""",false,false,"""N/A""","""false""","""2023-06-19T02:…","""1000"""
"""101732788""","""101732788""","""2023-06-19T03:…","""EDT""","""60""","""minute""",0.0,"""G""","""kCCF""","""N/A""",false,false,"""N/A""","""false""","""2023-06-19T03:…","""1000"""
"""101732788""","""101732788""","""2023-06-19T04:…","""EDT""","""60""","""minute""",0.0,"""G""","""kCCF""","""N/A""",false,false,"""N/A""","""false""","""2023-06-19T04:…","""1000"""
"""101732788""","""101732788""","""2023-06-19T05:…","""EDT""","""60""","""minute""",0.0,"""G""","""kCCF""","""N/A""",false,false,"""N/A""","""false""","""2023-06-19T05:…","""1000"""


In [302]:
def save_tsv_file_manifest(df, opco, phase, output_folder):
    rows_manifest = []
    for c in range(0, len(df), max_rows):
        date_str = datetime.now().strftime("%Y%m%d-%H%M%S%f")[:-3]
        tsv_filename = f'avangrid-energymanager-{opco}_intervalusage_{phase}_{date_str}.tsv'
        # Chunk the dataframe
        df_tmp = df[c:c + max_rows]
        full_path = os.path.join(output_folder, tsv_filename)
        
        df_tmp.write_csv(full_path, has_header=False, float_precision=3, separator='\t')
        
        # Append the manifest
        with open(full_path, 'rb') as f:
            # Check the size of the file
            f.seek(0, os.SEEK_END)
            file_size = f.tell()
            # Back to the pointer to the begining of the file
            f.seek(os.SEEK_SET)

            # Compute the MD5 of the file
            md5_checksum = hashlib.md5(f.read()).hexdigest()

            rows_manifest.append({
                "file_name": tsv_filename,
                "file_size": file_size,
                "md5_checksum": md5_checksum,
                "line_count": len(df_tmp)
            })

    date_str = datetime.now().strftime("%Y%m%d-%H%M%S%f")[:-3]
    manifest_file_name = f'avangrid-energymanager-{opco}_manifestmdm_{phase}_{date_str}.tsv'
    pd.DataFrame(rows_manifest).to_csv(os.path.join(output_folder, manifest_file_name), header=True, index=False, sep ='\t', lineterminator='\r\n')

In [303]:
def split_saved_tsv_files(df_final_, output_folder, divide_opcos=False, phase='uat', max_rows = 10_000_000):
    # phase = 'uat' # 'prod'
    # Group by opco
    if divide_opcos:
        for cocd, df_opco in df_final_.partition_by(by='CoCd', maintain_order=True, as_dict=True).items():
            opco = opco_dict[cocd]
            df_opco = df_opco.select(*list_final_order_file)
            save_tsv_file_manifest(df_opco, opco, phase, output_folder)
    else:
        df_opco = df_final_.select(*list_final_order_file)
        save_tsv_file_manifest(df_opco, 'ALL', phase, output_folder)

In [330]:
split_saved_tsv_files(df_final, 'output_gas', False)

In [331]:
df_final.head()

service_point_id,meter_id,timestamp,timezone,interval_value,interval_units,usage,energy_type,energy_units,energy_direction,is_estimate,is_outage,channel_id,is_deleted,update_datetime,CoCd
str,str,str,str,str,str,f64,str,str,str,bool,bool,str,str,str,str
"""101732788""","""101732788""","""2023-06-19T01:…","""EDT""","""60""","""minute""",0.0,"""G""","""kCCF""","""N/A""",false,false,"""N/A""","""false""","""2023-06-19T01:…","""1000"""
"""101732788""","""101732788""","""2023-06-19T02:…","""EDT""","""60""","""minute""",0.0,"""G""","""kCCF""","""N/A""",false,false,"""N/A""","""false""","""2023-06-19T02:…","""1000"""
"""101732788""","""101732788""","""2023-06-19T03:…","""EDT""","""60""","""minute""",0.0,"""G""","""kCCF""","""N/A""",false,false,"""N/A""","""false""","""2023-06-19T03:…","""1000"""
"""101732788""","""101732788""","""2023-06-19T04:…","""EDT""","""60""","""minute""",0.0,"""G""","""kCCF""","""N/A""",false,false,"""N/A""","""false""","""2023-06-19T04:…","""1000"""
"""101732788""","""101732788""","""2023-06-19T05:…","""EDT""","""60""","""minute""",0.0,"""G""","""kCCF""","""N/A""",false,false,"""N/A""","""false""","""2023-06-19T05:…","""1000"""


In [311]:
df_gas_nyseg.select('Meterid', 'ApprovedDate', 'CoCd').unique().join(df_final, left_on=['Meterid', 'CoCd'], right_on=['meter_id', 'CoCd']).head()

Meterid,ApprovedDate,CoCd,service_point_id,timestamp,timezone,interval_value,interval_units,usage,energy_type,energy_units,energy_direction,is_estimate,is_outage,channel_id,is_deleted,update_datetime
str,datetime[μs],str,str,str,str,str,str,f64,str,str,str,bool,bool,str,str,str


In [312]:
df_gas_rge.select('Meterid', 'ApprovedDate', 'CoCd').unique().join(df_final, left_on=['Meterid', 'CoCd'], right_on=['meter_id', 'CoCd']).head()

Meterid,ApprovedDate,CoCd,service_point_id,timestamp,timezone,interval_value,interval_units,usage,energy_type,energy_units,energy_direction,is_estimate,is_outage,channel_id,is_deleted,update_datetime
str,datetime[μs],str,str,str,str,str,str,f64,str,str,str,bool,bool,str,str,str
